In [190]:
import pandas as pd
import math

In [191]:
order_report=pd.read_excel("Company X - Order Report.xlsx")
pin_code=pd.read_excel("Company X - Pincode Zones.xlsx")
master=pd.read_excel("Company X - SKU Master.xlsx")
invoice=pd.read_excel("Courier Company - Invoice.xlsx")
rates=pd.read_excel("Courier Company - Rates.xlsx")

In [192]:
sku_order_report=pd.merge(order_report,master,on="SKU")

In [193]:
sku_order_report["total"]=sku_order_report["Order Qty"]*sku_order_report["Weight (g)"]

In [194]:
sku_order_report=sku_order_report.pivot_table(index="ExternOrderNo",values="total",aggfunc="sum")

In [195]:
sku_order_report["total"]=sku_order_report["total"]/1000

In [196]:
sku_order_report.reset_index(inplace=True)

In [197]:
sku_order_report=pd.merge(sku_order_report,invoice,left_on="ExternOrderNo",right_on="Order ID")

In [198]:
sku_order_report=pd.merge(sku_order_report,pin_code,on="Customer Pincode")

In [199]:
sku_order_report.drop_duplicates(inplace=True)

In [200]:
sku_order_report.reset_index(drop=True,inplace=True)

In [201]:
sku_order_report.drop(columns=['ExternOrderNo',
       'Warehouse Pincode_x', 'Customer Pincode', 'Warehouse Pincode_y'],inplace=True)

In [202]:
zone = {'a':'A', 'b':'B', 'c':'C', 'd':'D', 'e':'E'}
sku_order_report['Zone_x'] = sku_order_report['Zone_x'].map(zone)
sku_order_report['Zone_y'] = sku_order_report['Zone_y'].map(zone)

In [203]:
sku_order_report=pd.merge(sku_order_report,rates,left_on="Zone_y",right_on="Zone")

In [204]:
sku_order_report.drop(columns=["Zone"],inplace=True)

In [205]:
sku_order_report=pd.merge(sku_order_report,rates,left_on="Zone_x",right_on="Zone")

In [206]:
sku_order_report.drop(columns=["Zone"],inplace=True)

In [207]:
def calculate(zone, weight, shipment):
    zone = rates[rates['Zone'] == zone]
    slab = zone['Weight Slabs'].values[0]
    
    if shipment == 'Forward charges':
        if weight > slab:
            fixed = weight - slab
            additional = math.ceil(fixed / slab)
            charge = zone['Forward Fixed Charge'].values[0] + zone['Forward Additional Weight Slab Charge'].values[0]*additional
        else:
            charge = zone['Forward Fixed Charge'].values[0]
    else:
        if weight > slab:
            fixed = weight - slab
            additional = math.ceil(fixed / slab)
            charge = zone['Forward Fixed Charge'].values[0] + zone['Forward Additional Weight Slab Charge'].values[0]*additional + zone['RTO Fixed Charge'].values[0] + zone['RTO Additional Weight Slab Charge'].values[0]*additional
        else:
            charge = zone['Forward Fixed Charge'].values[0] + zone['RTO Fixed Charge'].values[0]
    
    return charge

In [208]:
sku_order_report['Expected Charge as per X(Rs.)'] = sku_order_report.apply(lambda row: calculate(row['Zone_y'], row['total'], row['Type of Shipment']), axis=1)

In [209]:
sku_order_report.drop(columns=["Forward Fixed Charge_y","Forward Additional Weight Slab Charge_y","RTO Fixed Charge_y","RTO Additional Weight Slab Charge_y"],axis=1,inplace=True)

In [210]:
sku_order_report.rename(columns={"total":"Total Weight as per X(KG)","Zone_y":"Delivery Zone as per X"
,"Zone_x":"Delivery Zone charged by Courier Company","Weight Slabs_x":"Weight Slab as per X(Kg)","Weight Slabs_y":"Weight slab charged by Courier Company (KG)"
,'Charged Weight':' Total weight as per Courier Company(KG)','Billing Amount (Rs.)':'Charges Billed by Courier Company(Rs.)'
},inplace=True)

In [215]:
sku_order_report = sku_order_report[['Order ID', 'AWB Code','Total Weight as per X(KG)','Weight Slab as per X(Kg)',
       ' Total weight as per Courier Company(KG)','Weight slab charged by Courier Company (KG)','Delivery Zone as per X',
       'Delivery Zone charged by Courier Company', 'Expected Charge as per X(Rs.)',
       'Charges Billed by Courier Company(Rs.)']]

In [216]:
sku_order_report

,Order ID,AWB Code,Total Weight as per X(KG),Weight Slab as per X(Kg),Total weight as per Courier Company(KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X(Rs.),Charges Billed by Courier Company(Rs.)
0,2001806210,1091117221940,0.220,1.0,2.92,1.0,B,B,33.0,174.5
1,2001806226,1091117222065,0.480,1.5,0.68,1.5,D,D,45.4,90.2
2,2001806229,1091117222080,0.500,1.5,0.71,1.5,D,D,45.4,90.2
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,45.4,135.0
4,2001806233,1091117222135,0.245,1.0,0.78,1.0,B,B,33.0,61.3
...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1.0,0.50,1.5,B,D,33.0,45.4
120,2001822466,1091121305541,1.376,1.0,1.10,1.5,B,D,61.3,135.0
121,2001823564,1091121666133,0.672,1.5,0.70,1.5,D,D,86.7,172.8
122,2001825261,1091121981575,1.557,1.5,1.60,1.5,D,D,176.3,345.0


In [217]:
sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] = round(sku_order_report['Expected Charge as per X(Rs.)'] - sku_order_report['Charges Billed by Courier Company(Rs.)'],2)

C:\Users\j s\AppData\Local\Temp\ipykernel_4708\295135584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] = round(sku_order_report['Expected Charge as per X(Rs.)'] - sku_order_report['Charges Billed by Courier Company(Rs.)'],2)


In [218]:
zero_diff_count = (sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0.0).sum()
zero_diff_amount = (sku_order_report[sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0.0]['Charges Billed by Courier Company(Rs.)']).sum()

In [219]:
overcharge_count = (sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0.0).sum()
overcharge_amount = round((sku_order_report[sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0.0]['Difference Between Expected Charges and Billed Charges (Rs.)']).sum(),2)

In [220]:
undercharge_count = (sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0.0).sum()
undercharge_amount = round((sku_order_report[sku_order_report['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0.0]['Difference Between Expected Charges and Billed Charges (Rs.)']).sum(),2)

In [221]:
summary_table = {' ':['Total orders where X has been correctly charged','Total Orders where X has been overcharged','Total Orders where X has been undercharged '],
                 'Count': [zero_diff_count,overcharge_count,undercharge_count],
                 'Amount (Rs.)': [zero_diff_amount,overcharge_amount,undercharge_amount]}

In [222]:
summary_table = pd.DataFrame(summary_table)

In [224]:
with pd.ExcelWriter('Result.xlsx', engine='xlsxwriter') as writer:
    
    summary_table.to_excel(writer, sheet_name='Summary', index=False)   
    sku_order_report.to_excel(writer, sheet_name='Calculations', index=False)

In [225]:
print('Completed')

Completed


In [226]:
sku_order_report

,Order ID,AWB Code,Total Weight as per X(KG),Weight Slab as per X(Kg),Total weight as per Courier Company(KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X(Rs.),Charges Billed by Courier Company(Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,1.0,2.92,1.0,B,B,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,1.5,0.68,1.5,D,D,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,1.5,0.71,1.5,D,D,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,45.4,135.0,-89.6
4,2001806233,1091117222135,0.245,1.0,0.78,1.0,B,B,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,1.0,0.50,1.5,B,D,33.0,45.4,-12.4
120,2001822466,1091121305541,1.376,1.0,1.10,1.5,B,D,61.3,135.0,-73.7
121,2001823564,1091121666133,0.672,1.5,0.70,1.5,D,D,86.7,172.8,-86.1
122,2001825261,1091121981575,1.557,1.5,1.60,1.5,D,D,176.3,345.0,-168.7
